# Imports

In [ ]:
# Python version: 3.11.0
# matplotlib==3.10.0
# pandas==2.2.3
# numpy==2.2.1
# scipy==1.15.0
# scikit-learn==1.6.1
# seaborn==0.13.2

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Graph Stats

In [ ]:
# Graph statistics files
graph_stat_files = {"./graph_hdagg_metis.txt", "./graph_florida_stats.txt", "./graph_erdos_renyi_stats.txt", "./graph_band_stats.txt", "./graph_iChol_stats.txt"}

In [ ]:
# Reading files into pandas dataframe
graph_df = pd.concat( [pd.read_csv(file) for file in graph_stat_files ], ignore_index=True )

In [ ]:
# Dropping duplicates
graph_df.drop_duplicates(inplace = True)

In [ ]:
# Changing index to Graph
graph_df = graph_df.set_index("Graph", drop=True)

In [ ]:
# Adding double precision floating point operations to the dataframe
graph_df["FLOP_double_precision"] = 2 * graph_df["Edges"] + graph_df["Vertices"]

In [ ]:
# Adding number of non-zeroes to the dataframe
graph_df["Number_of_non-zeroes"] = graph_df["Edges"] + graph_df["Vertices"]

## Graph Filter

In [ ]:
# Graph filter templates

# All graphs
all_graphs = set(graph_df.index)

# SPAA and AI graphs
spaa_ai_graphs = set( [g for g in all_graphs if g[:4] == "inst" ] )

# Erdos-Renyi graphs
erdos_renyi_graphs = set( [g for g in all_graphs if g[:5] == "Erdos" ] )

# Random Bandwidth graphs
random_band_graphs = set( [g for g in all_graphs if g[:10] == "RandomBand" ] )

# Florida graphs
florida_graphs = set( [g for g in all_graphs if g[:10] != "RandomBand" and g[:5] != "Erdos" and g[:4] != "inst"])


In [ ]:
# Setting graph filters for subsequent SpTrSV data analysis
graph_subset = florida_graphs


# SpTrSV Data

In [ ]:
# Folder location
folder_location_x86 = "./SpTrSV_Data/plot_22_x86_SC/"
folder_location_ARM = "./SpTrSV_Data/plot_22_ARM/"
folder_location_epyc = "./SpTrSV_Data/plot_22_epyc/"

In [ ]:
# Listing all files in folder_location
data_files_x86 = set([file for file in os.listdir(folder_location_x86)])
data_files_x86 
data_files_x86 = [file for file in data_files_x86 if file[:3] != "log"]

# Listing all files in folder_location
data_files_ARM = set([file for file in os.listdir(folder_location_ARM)])
data_files_ARM 
data_files_ARM = [file for file in data_files_ARM if file[:3] != "log"]

# Listing all files in folder_location
data_files_epyc = set([file for file in os.listdir(folder_location_epyc)])
data_files_epyc 
data_files_epyc = [file for file in data_files_epyc if file[:3] != "log"]

In [ ]:
# Specifying Datatypes
data_type_dic = {
    "Graph":                            "object",
    "Machine":                          "object",
    "Algorithm":                        "object",
    "Permutation":                      "object",
    "SpTrSV_Runtime":                  "float64",
    "Work_Cost":                         "int64",
    "Base_Comm_Cost":                    "int64",
    "Supersteps":                        "int64",
    "_Base_Buffered_Sending":            "int64",
    "Base_CostsTotalCommunication":    "float64",
    "Schedule_Compute_time":             "int64",
    "Processors":                        "int64",
    "BSP_g":                             "int64",
    "BSP_l":                             "int64",
    "Architecture":                     "object",
}

data_default_na_val_dic = {
    "Graph":                              "",
    "Machine":                            "",
    "Algorithm":                          "",
    "Permutation":                        "",
    "SpTrSV_Runtime":                  "0.0",
    "Work_Cost":                         "0",
    "Base_Comm_Cost":                    "0",
    "Supersteps":                        "1",
    "_Base_Buffered_Sending":            "0",
    "Base_CostsTotalCommunication":    "0.0",
    "Schedule_Compute_time":             "1",
    "Processors":                        "0",
    "BSP_g":                             "0",
    "BSP_l":                             "0",
    "Architecture":                       "",
}

In [ ]:
# Reading files into pandas dataframe
SpTrSV_df_x86 = pd.concat( [pd.read_csv( folder_location_x86 + file) for file in data_files_x86], ignore_index=True )
SpTrSV_df_x86 = SpTrSV_df_x86.assign(Architecture="x86")

SpTrSV_df_ARM = pd.concat( [pd.read_csv( folder_location_ARM + file) for file in data_files_ARM], ignore_index=True )
SpTrSV_df_ARM = SpTrSV_df_ARM.assign(Architecture="ARM")

SpTrSV_df_epyc = pd.concat( [pd.read_csv( folder_location_epyc + file) for file in data_files_epyc], ignore_index=True )
SpTrSV_df_epyc = SpTrSV_df_epyc.assign(Architecture="epyc")

SpTrSV_df = pd.concat( [SpTrSV_df_x86, SpTrSV_df_ARM, SpTrSV_df_epyc], ignore_index=True )


In [ ]:
# Data Deleting folder structure from 'Graph' column
SpTrSV_df["Graph"] = SpTrSV_df["Graph"].str.split("/").str[-1]

In [ ]:
# Adding BSP parameters to the dataframe
SpTrSV_df[["Processors", "BSP_g", "BSP_l"]] = SpTrSV_df["Machine"].str.split("_", n=2, expand=True).reindex(range(3), axis=1)
SpTrSV_df["Processors"] = SpTrSV_df["Processors"].astype("object").str.slice(start=1).astype("int64", errors="ignore")
SpTrSV_df["BSP_g"] = SpTrSV_df["BSP_g"].astype("object").str.slice(start=1).astype("int64", errors="ignore")
SpTrSV_df["BSP_l"] = SpTrSV_df["BSP_l"].astype("object").str.slice(start=1).astype("int64", errors="ignore")

In [ ]:
# Casting to correct Datatypes
for key, val in data_type_dic.items():
    SpTrSV_df[key] = SpTrSV_df[key].fillna(data_default_na_val_dic[key]).astype(val)

In [ ]:
# Function to compute giga double precision floating point operations, denoted by GFP64
def compute_GFP64(time, graph, df = graph_df):
    flop = df.at[graph ,"FLOP_double_precision"]
    return (flop / time) / 1000000000

In [ ]:
# Adding giga double precision floating point operations, denoted by GFP64, to the dataframe
SpTrSV_df["GFP64/s"] = SpTrSV_df[["Graph", "SpTrSV_Runtime"]].apply(lambda x: compute_GFP64(x["SpTrSV_Runtime"], x["Graph"]), axis=1)

In [ ]:
# Set schedule compute time at least 1ms
SpTrSV_df["Schedule_Compute_time"] = SpTrSV_df["Schedule_Compute_time"].replace(0,1)

In [ ]:
# Data sorting
SpTrSV_df.sort_values([ "Graph", "Algorithm" ], axis=0, inplace=True)

## Filters (incl. Algorithm, Processor)

In [ ]:
# Setting algorithm filter
alg_filter_set = set(SpTrSV_df["Algorithm"].unique())
alg_filter_set = set(["HDAGG_BIN", "SpMP", "SMGreedyBspGrowLocalAutoCoresParallel"])
                   

# Must alway contain Serial and HD_original
alg_filter_set.add("Serial")

In [ ]:
# Setting processor filter
proc_filter = SpTrSV_df["Processors"].unique()

In [ ]:
# Applying algorithm filter
SpTrSV_df_filtered = SpTrSV_df[ SpTrSV_df["Algorithm"].isin(alg_filter_set) ]

In [ ]:
# Applying graph filter
SpTrSV_df_filtered = SpTrSV_df_filtered[ SpTrSV_df_filtered["Graph"].isin(graph_subset) ]


In [ ]:
# Applying processor filter
SpTrSV_df_filtered = SpTrSV_df_filtered[ SpTrSV_df_filtered["Processors"].isin(proc_filter) ]

## Evaluation

In [ ]:
# Setting up new pandas dataframe with geometric mean of GFP64 and speed-up over serial execution
geom_mean_FLOPS_df = pd.DataFrame(columns=["Processors", "Graphs", "Architecture", "Algorithm", "GFP64/s", "Speedup_over_Serial"])
for name, group in SpTrSV_df_filtered.groupby(["Processors", "Graph", "Architecture"]):
    serial_flops = np.exp( np.log(group[ group["Algorithm"] == "Serial" ]["GFP64/s"]).mean() )
    
    for alg, alg_group in group.groupby("Algorithm"):
        flops = np.exp( np.log(alg_group["GFP64/s"]).mean() )
        alg_schedule_compute_time = alg_group["Schedule_Compute_time"].mean()
        alg_supersteps = alg_group["Supersteps"].mean()
        temp_df = pd.DataFrame( [[name[0], name[1], name[2], alg, flops, flops/serial_flops]], columns=["Processors", "Graphs", "Architecture", "Algorithm", "GFP64", "Speedup_over_Serial", ] )
        geom_mean_FLOPS_df = pd.concat([geom_mean_FLOPS_df, temp_df], ignore_index=True)

In [ ]:
# Adding logarithm of speed-up over serial and HDagg
geom_mean_FLOPS_df["Log2_speedup_over_Serial"] = np.log2( geom_mean_FLOPS_df["Speedup_over_Serial"] )

### GFP64/s

In [ ]:
# Average Log speed-ups over Serial
###########################################################
#####  The output of this cell corresponds to Table 7.4
###########################################################
florida_agg = geom_mean_FLOPS_df[["Processors", "Algorithm","Architecture", "Log2_speedup_over_Serial"]].groupby(["Processors","Architecture","Algorithm",]).mean()
florida_agg["Geommean"] = np.exp2(florida_agg["Log2_speedup_over_Serial"])
florida_agg